In [ ]:
import logging

# Set the logging level for semantic_kernel to DEBUG to see detailed request information
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.getLogger().setLevel(logging.DEBUG)

# Enable debug logging specifically for semantic kernel components
logging.getLogger("semantic_kernel").setLevel(logging.DEBUG)
logging.getLogger("semantic_kernel.connectors.ai.ollama").setLevel(logging.DEBUG)
logging.getLogger("semantic_kernel.connectors.mcp").setLevel(logging.DEBUG)

# Enable HTTP-level logging to see actual POST request bodies
logging.getLogger("httpx").setLevel(logging.DEBUG)
logging.getLogger("httpcore").setLevel(logging.DEBUG)
logging.getLogger("urllib3").setLevel(logging.DEBUG)

# Also enable aiohttp if it's being used
logging.getLogger("aiohttp").setLevel(logging.DEBUG)

In [ ]:
# import json
# import httpx
# from functools import wraps

# # Store the original request method BEFORE we replace it
# _original_request = httpx.AsyncClient.request

# async def logged_request(self, method, url, **kwargs):
#     print(f"\n=== HTTP REQUEST ===")
#     print(f"Method: {method}")
#     print(f"URL: {url}")
    
#     # Log headers
#     headers = kwargs.get('headers', {})
#     print(f"Headers: {dict(headers) if headers else 'None'}")
    
#     # Log request body
#     content = kwargs.get('content')
#     json_data = kwargs.get('json')
#     data = kwargs.get('data')
    
#     if content:
#         try:
#             # Try to parse as JSON for pretty printing
#             if isinstance(content, (str, bytes)):
#                 content_str = content.decode() if isinstance(content, bytes) else content
#                 parsed = json.loads(content_str)
#                 print(f"Request Body (JSON):\n{json.dumps(parsed, indent=2)}")
#             else:
#                 print(f"Request Body (Raw): {content}")
#         except:
#             print(f"Request Body (Raw): {content}")
#     elif json_data:
#         print(f"Request Body (JSON):\n{json.dumps(json_data, indent=2)}")
#     elif data:
#         print(f"Request Body (Data): {data}")
#     else:
#         print("Request Body: None")
    
#     print(f"=== END REQUEST ===\n")
    
#     # Call the ACTUAL original method, not the monkey-patched one
#     return await _original_request(self, method, url, **kwargs)

# # Apply the monkey patch
# httpx.AsyncClient.request = logged_request
# print("HTTP request logging enabled!")

In [ ]:
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

mcp_server = MCPStreamableHttpPlugin(
                    name="test",
                    url="http://localhost:8001/mcp",
                )

try:
    await mcp_server.connect()
    available_tools = await mcp_server.session.list_tools()
    logging.info(f"Successfully connected to MCP server: <{mcp_server.name}> with tools {available_tools}")
    # await mcp_server.close()
except Exception as e:
    logging.error(f"Error connecting to {mcp_server.name} MCP server: {e}")

In [ ]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings





# Initialize the kernel
kernel = Kernel()

# Add Azure OpenAI chat completion
chat_completion = OllamaChatCompletion(
    ai_model_id="gpt-oss:20b",
    host="http://ollama.home",
)

kernel.add_service(chat_completion)

# Set the logging level for  semantic_kernel.kernel to DEBUG.
setup_logging()

kernel.add_plugin(mcp_server)

# Enable planning
execution_settings = OllamaChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()

# Initiate a back-and-forth chat
userInput = None
while True:
    # Collect user input
    userInput = input("User > ")

    # Terminate the loop if the user says "exit"
    if userInput == "exit":
        break

    # Add user input to the history
    history.add_user_message(userInput)

    # Get the response from the AI
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    # Print the results
    print("Assistant > " + str(result))

    # Add the message from the agent to the chat history
    history.add_message(result)
